In [ ]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import requests
import time
import hmac
import hashlib
import base64

# Signature 클래스: API 서명 생성
class Signature:
    @staticmethod
    def generate(timestamp, method, uri, secret_key):
        message = "{}.{}.{}".format(timestamp, method, uri)
        hash = hmac.new(bytes(secret_key, "utf-8"), bytes(message, "utf-8"), hashlib.sha256)
        return base64.b64encode(hash.digest()).decode("utf-8")

# API 요청을 위한 헤더를 가져오는 함수
def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = Signature.generate(timestamp, method, uri, secret_key)
    return {
        'Content-Type': 'application/json; charset=UTF-8',
        'X-Timestamp': timestamp,
        'X-API-KEY': api_key,
        'X-Customer': str(customer_id),
        'X-Signature': signature
    }

# 네이버 API에서 연관 키워드 결과를 가져오는 함수
def get_results(hintKeywords):
    BASE_URL = 'https://api.naver.com'
    API_KEY = '010000000013d50306951198c56725e6d082916ed4870b354963d7e7119eb766144d8c58c0'  # 실제 API_KEY로 변경하세요
    SECRET_KEY = 'AQAAAAAT1QMGlRGYxWcl5tCCkW7UFW5GoTbNMAkwsweVpmyjWw=='  # 실제 SECRET_KEY로 변경하세요
    CUSTOMER_ID = '2496501'  # 실제 CUSTOMER_ID로 변경하세요

    uri = '/keywordstool'
    method = 'GET'

    params = {
        'hintKeywords': hintKeywords,
        'showDetail': '1'
    }

    r = requests.get(BASE_URL + uri, params=params,
                     headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))

    if r.status_code == 200:
        return pd.DataFrame(r.json()['keywordList'])
    else:
        messagebox.showerror("Error", f"API Error: {r.status_code}, {r.text}")
        return None

# 연관 키워드로 결과 리스트박스 업데이트 함수
def update_related_keywords():
    hintKeywords = keyword_entry.get()  # 키워드 입력 필드에서 입력 가져오기
    if not hintKeywords:
        messagebox.showwarning("입력 에러", "키워드를 입력하세요.")
        return
    results_df = get_results(hintKeywords)

    if results_df is not None:
        # API 응답의 컬럼 이름 확인
        print("API 응답 컬럼:", results_df.columns)

        # 이제 'relKeyword'를 정확한 컬럼 이름으로 사용
        if 'relKeyword' in results_df.columns:
            related_keywords_listbox.delete(0, tk.END)  # 이전 결과 지우기
            for keyword in results_df['relKeyword']:
                related_keywords_listbox.insert(tk.END, keyword)
        else:
            messagebox.showerror("에러", "API 응답에 예상되는 'relKeyword' 컬럼이 없습니다.")



# 제외 단어를 엑셀 파일에 저장하는 함수
def save_excluded_words():
    excluded_words_input = exclude_entry.get()  # 제외 단어 입력 필드에서 입력 가져오기
    if excluded_words_input:
        excluded_words_list = [word.strip() for word in excluded_words_input.split(',')] # 쉼표로 구분된 단어들을 리스트로 만듦, strip()으로 공백 제거
        df = pd.DataFrame({'제외 단어': excluded_words_list}) # 컬럼 이름을 '제외 단어'로 설정
        try:
            df.to_excel("제외단어.xlsx", index=False) # 엑셀 파일로 저장, index 제거
            messagebox.showinfo("저장됨", "제외 단어가 엑셀 파일로 저장되었습니다 (제외단어.xlsx).")
        except Exception as e:
            messagebox.showerror("저장 에러", f"엑셀 파일 저장 중 오류 발생: {e}")
        exclude_entry.delete(0, tk.END)  # 제외 단어 입력 필드 지우기
    else:
        messagebox.showwarning("입력 에러", "제외할 단어를 입력하세요.")


# 입력을 기반으로 상세 키워드 업데이트 함수
def update_detailed_keywords():
    # search_term = detailed_keywords_entry.get()  # 입력 필드에서 입력 가져오기 # 입력창 제거로 인해 필요 없어짐
    search_term = "" # 입력창 제거로 인해 항상 빈 문자열로 검색하도록 변경

    if not search_term: # 입력창이 없으므로 항상 이 조건은 참이 됨. 필요에 따라 로직 변경 고려
        pass # 입력창이 없으므로, 항상 모든 키워드를 보여주도록 변경하려면 이 부분을 수정해야 함. 현재는 입력 없어도 검색 기능은 작동하도록 둠.
        # messagebox.showwarning("입력 에러", "검색어를 입력하세요.") # 입력창 제거로 인해 경고 메시지 불필요
        # return # 입력창 제거로 인해 return 불필요

    # 연관 키워드 목록을 반복하여 일치하는 항목 찾기
    detailed_keywords_listbox.delete(0, tk.END)  # 이전 결과 지우기

    for keyword in related_keywords_listbox.get(0, tk.END):  # 모든 연관 키워드 반복
        if search_term in keyword: # 입력창 제거로 인해 항상 모든 키워드를 포함하는 것으로 처리됨.
            detailed_keywords_listbox.insert(tk.END, keyword)  # 일치하는 키워드를 리스트박스에 추가

    if detailed_keywords_listbox.size() == 0:
        messagebox.showinfo("일치하는 항목 없음", f"'{search_term}'을 포함하는 키워드가 없습니다.") # 입력창 제거로 인해 이 메시지는 거의 표시되지 않을 것임. 필요에 따라 메시지 변경 또는 제거 고려


# 메인 키워드 리스트박스 업데이트 함수
def update_main_keywords():
    main_keywords = main_keywords_entry.get()  # 메인 키워드 입력 필드에서 입력 가져오기
    if main_keywords:
        main_keywords_listbox.insert(tk.END, main_keywords)
        main_keywords_entry.delete(0, tk.END)  # 입력 필드 지우기

# 메인 윈도우 생성
root = tk.Tk()
root.title("키워드 도구")
root.geometry("1200x800") # 가로 길이 늘림

# 위젯 생성 및 배치

# --- Column 0 (Leftmost) ---
keyword_label = tk.Label(root, text="키워드 입력:")
keyword_label.grid(row=0, column=0, padx=5, pady=5, sticky='w')
keyword_entry = tk.Entry(root, width=30)
keyword_entry.grid(row=1, column=0, padx=5, pady=5, sticky='ew')
search_button = tk.Button(root, text="연관 키워드 검색", command=update_related_keywords)
search_button.grid(row=2, column=0, padx=5, pady=5, sticky='ew')

exclude_label = tk.Label(root, text="제외 단어 입력:")
exclude_label.grid(row=3, column=0, padx=5, pady=5, sticky='w')
exclude_entry = tk.Entry(root, width=30)
exclude_entry.grid(row=4, column=0, padx=5, pady=5, sticky='ew')
save_exclude_button = tk.Button(root, text="제외 단어 저장", command=save_excluded_words)
save_exclude_button.grid(row=5, column=0, padx=5, pady=5, sticky='ew')

add_detailed_keywords_button = tk.Button(root, text="상세 키워드 추가", command=update_detailed_keywords)
add_detailed_keywords_button.grid(row=6, column=0, padx=5, pady=5, sticky='ew')


# --- Column 1 (Center-Left) ---
related_keywords_label = tk.Label(root, text="연관 키워드:")
related_keywords_label.grid(row=0, column=1, padx=5, pady=5, sticky='w')
related_keywords_listbox = tk.Listbox(root, width=30, height=20)
related_keywords_listbox.grid(row=1, column=1, rowspan=7, padx=5, pady=5, sticky='nsew')

# --- Column 2 (Center-Right) ---
detailed_keywords_label = tk.Label(root, text="상세 키워드:")
detailed_keywords_label.grid(row=0, column=2, padx=5, pady=5, sticky='w')
detailed_keywords_listbox = tk.Listbox(root, width=30, height=20)
detailed_keywords_listbox.grid(row=1, column=2, rowspan=7, padx=5, pady=5, sticky='nsew')

# --- Column 3 (Rightmost) ---
main_keywords_label = tk.Label(root, text="메인 키워드:")
main_keywords_label.grid(row=0, column=3, padx=5, pady=5, sticky='w')
main_keywords_listbox = tk.Listbox(root, width=30, height=15)
main_keywords_listbox.grid(row=1, column=3, rowspan=6, padx=5, pady=5, sticky='nsew')
main_keywords_entry = tk.Entry(root, width=30)
main_keywords_entry.grid(row=7, column=3, padx=5, pady=5, sticky='ew')
add_main_keywords_button = tk.Button(root, text="메인 키워드 추가", command=update_main_keywords)
add_main_keywords_button.grid(row=8, column=3, padx=5, pady=5, sticky='ew')

# Configure column and row weights for resizing
root.columnconfigure(1, weight=1) # 연관 키워드 column expands
root.columnconfigure(2, weight=1) # 상세 키워드 column expands
root.columnconfigure(3, weight=1) # 메인 키워드 column expands
root.rowconfigure(1, weight=1)    # Make row 1 (where listboxes are) expandable


print("테스트 1")

# Tkinter 이벤트 루프 실행
root.mainloop()

테스트 1
API 응답 컬럼: Index(['relKeyword', 'monthlyPcQcCnt', 'monthlyMobileQcCnt',
       'monthlyAvePcClkCnt', 'monthlyAveMobileClkCnt', 'monthlyAvePcCtr',
       'monthlyAveMobileCtr', 'plAvgDepth', 'compIdx'],
      dtype='object')
